In [ ]:
# @title Mount drive và import pandas
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd

Mounted at /content/drive


# Chuẩn bị một số hàm thường dùng

In [ ]:
# @title Hàm **url_parse**: unquote URL
# @markdown Chuyển các giá trị như '%20' về thành ' '

from urllib.parse import unquote

def url_parse(query):
    query = unquote(query)
    return query

In [ ]:
# @title Hàm **newremove**: loại bỏ các giá trị '\n'

def newremove(statement):
  return statement.replace('\n','')

In [ ]:
# @title Hàm **whiteremove**: loại bỏ khoảng trống trong url
def whiteremove(statement):
  return statement.replace(' ','')

In [ ]:
# @title Hàm **plusreplace**: thay dấu + thành khoảng trống

def plusreplace(statement):
  return statement.replace('+',' ')

# Thực hiện tạo dataset SQLi

In [ ]:
# @title Đọc stage.csv vào DataFrame df và thêm cột label

df = pd.read_csv("/content/drive/MyDrive/Machine_Log_Analysis/Dataset/stage.csv")
df = df[['type','url']]
df['label'] = 0
df


,type,url,label
0,Valid,/l_t@/_Feu1wvhtpass2/1nieQnnrvnzktuasain/tg1AR...,0
1,Valid,/inssgtz7ieltdSstbw7e/neQhmsdwu7imdb0etet/eT/h...,0
2,Valid,/fonRnt/7N.D-4BRSsXb@TU/qgheW.cfm,0
3,Valid,/dyylkL.XD9cPu/4Ot0ta/ts6xNrP1/hssh/a2cuerht/s...,0
4,Valid,/2m6VLb1r37jSPC/cWVv/Mbar/oqrd0/msc/etceebwgi/...,0
...,...,...,...
125971,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125972,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125973,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125974,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0


In [ ]:
# @title Gán nhãn label cho các hàng SQLi và chuẩn hóa URL

# @markdown - Thực hiện:
# @markdown - Gán nhãn những hàng có type = SQLi với label = 1
# @markdown - Fill những ô trống bằng NULL
# @markdown - Chuẩn hóa url: unquote, bỏ các giá trị newline và đưa tất cả về chữ thường

df.loc[df['type']=='SQLi','label'] = 1
df = df.fillna('NULL')

df['url'] = df['url'].apply(url_parse)
df['url'] = df['url'].apply(newremove)
df['url'] = df['url'].str.lower()

df

,type,url,label
0,Valid,/l_t@/_feu1wvhtpass2/1nieqnnrvnzktuasain/tg1ar...,0
1,Valid,/inssgtz7ieltdsstbw7e/neqhmsdwu7imdb0etet/et/h...,0
2,Valid,/fonrnt/7n.d-4brssxb@tu/qghew.cfm,0
3,Valid,/dyylkl.xd9cpu/4ot0ta/ts6xnrp1/hssh/a2cuerht/s...,0
4,Valid,/2m6vlb1r37jspc/cwvv/mbar/oqrd0/msc/etceebwgi/...,0
...,...,...,...
125971,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125972,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125973,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125974,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0


In [ ]:
# @title Gán thuộc tính cho từng dòng

# @markdown Tạo một list chứa các thuộc tính. Thêm các thuộc tính này thành các cột trong DataFrame sau đó gán nhãn 1 cho những dòng có url chứa thuộc tính tương ứng


ls = ["-", "/**/", "%", "+", "'", ";", "#", "=", "[", "]", "(", ")", "^", "*", "char",
              ",", "-", "<", ">", ".", "|", "\"", "<>", "<=", ">=", "&&", "||", ":", "!=", "count",
              "into", "or", "and", "not", "null", "select", "union", "#", "insert", "update", "delete",
              "drop", "replace", "all", "any", "from", "count", "user", "where", "sp", "xp", "like",
              "exec", "admin", "table", "sleep", "commit", "()", "between"]

attr = set(ls)
len(attr)

for a in attr:
  df[a] = 0

def generate(statement,col):
    if col in str(statement):
        return 1
    else:
        return 0

for col in attr:
  df[col] = df['url'].apply(generate, col=col)

df

,type,url,label,"""",xp,all,=,any,+,>,...,:,like,-,union,(,char,!=,&&,update,commit
0,Valid,/l_t@/_feu1wvhtpass2/1nieqnnrvnzktuasain/tg1ar...,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,Valid,/inssgtz7ieltdsstbw7e/neqhmsdwu7imdb0etet/et/h...,0,0,1,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,Valid,/fonrnt/7n.d-4brssxb@tu/qghew.cfm,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Valid,/dyylkl.xd9cpu/4ot0ta/ts6xnrp1/hssh/a2cuerht/s...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Valid,/2m6vlb1r37jspc/cwvv/mbar/oqrd0/msc/etceebwgi/...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125971,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125972,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125973,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125974,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# @title Xuất df ra thành final_SQLi.csv

import csv
df.to_csv('/content/drive/MyDrive/Machine_Log_Analysis/Dataset/Final_Csv/final_SQLi.csv', escapechar='\\', quoting=csv.QUOTE_MINIMAL)

# Thực hiện tạo dataset XSS

In [ ]:
# @title Đọc stage.csv vào DataFrame df và thêm cột label


df = pd.read_csv("/content/drive/MyDrive/Machine_Log_Analysis/Dataset/stage.csv")
df = df[['type','url']]
df['label'] = 0
df

In [ ]:
# @title Gán nhãn label cho các hàng XSS và chuẩn hóa URL

# @markdown - Thực hiện:
# @markdown - Gán nhãn những hàng có type = XSS với label = 1
# @markdown - Fill những ô trống bằng NULL
# @markdown - Chuẩn hóa url: unquote, bỏ các giá trị newline, khoảng trống và đưa tất cả về chữ thường


df.loc[df['type']=='XSS','label'] = 1
df = df.fillna('NULL')

df['url'] = df['url'].apply(url_parse)
df['url'] = df['url'].apply(newremove)
df['url'] = df['url'].apply(whiteremove)

df['url'] = df['url'].str.lower()

In [1]:
# @title Gán thuộc tính cho từng dòng

# @markdown Tạo một list chứa các thuộc tính. Thêm các thuộc tính này thành các cột trong DataFrame sau đó gán nhãn 1 cho những dòng có url chứa thuộc tính tương ứng

ls = ["&", "%", "/", "\\", "+", "'", "?", "!", ";", "#", "=", "[", "]", "$",
      "(", ")", "^", "*", ",", "-", "<", ">", "@", "_", ":", "{", "}", " ",
      ".", " ", "|", "\"", "<>", "'", "<>", "[]", "==", "&#", "document",
      "window", "iframe", "location", "this", "onload", "onerror",
      "createelement", "string.fromcharcode", "search", "div", "img",
      "<script", "src", "href", "cookie", "var", "eval()", "http", ".js",]

attr = set(ls)
len(attr)

for a in attr:
  df[a] = 0

def generate(statement,col):
    if col in str(statement):
        return 1
    else:
        return 0

for col in attr:
  df[col] = df['url'].apply(generate, col=col)

df

NameError: name 'df' is not defined

In [ ]:
# @title Xuất df ra thành final_XSS.csv

import csv
df.to_csv('/content/drive/MyDrive/Machine_Log_Analysis/Dataset/Final_Csv/final_XSS.csv', escapechar='\\', quoting=csv.QUOTE_MINIMAL)

# Thực hiện tạo dataset Path Traversal

In [ ]:
# @title Đọc stage.csv vào DataFrame df và thêm cột label

df = pd.read_csv("/content/drive/MyDrive/Machine_Log_Analysis/Dataset/stage.csv")
df = df[['type','url']]
df['label'] = 0
df


,type,url,label
0,Valid,/l_t@/_Feu1wvhtpass2/1nieQnnrvnzktuasain/tg1AR...,0
1,Valid,/inssgtz7ieltdSstbw7e/neQhmsdwu7imdb0etet/eT/h...,0
2,Valid,/fonRnt/7N.D-4BRSsXb@TU/qgheW.cfm,0
3,Valid,/dyylkL.XD9cPu/4Ot0ta/ts6xNrP1/hssh/a2cuerht/s...,0
4,Valid,/2m6VLb1r37jSPC/cWVv/Mbar/oqrd0/msc/etceebwgi/...,0
...,...,...,...
125971,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125972,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125973,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125974,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0


In [ ]:
# @title Gán nhãn label cho các hàng PT và chuẩn hóa URL

# @markdown - Thực hiện:
# @markdown - Gán nhãn những hàng có type = PT với label = 1
# @markdown - Fill những ô trống bằng NULL
# @markdown - Chuẩn hóa url: unquote, bỏ các giá trị newline và đưa tất cả về chữ thường

df.loc[df['type']=='PT','label'] = 1
df = df.fillna('NULL')

df['url'] = df['url'].apply(url_parse)
df['url'] = df['url'].apply(newremove)
df['url'] = df['url'].str.lower()

df

,type,url,label
0,Valid,/l_t@/_feu1wvhtpass2/1nieqnnrvnzktuasain/tg1ar...,0
1,Valid,/inssgtz7ieltdsstbw7e/neqhmsdwu7imdb0etet/et/h...,0
2,Valid,/fonrnt/7n.d-4brssxb@tu/qghew.cfm,0
3,Valid,/dyylkl.xd9cpu/4ot0ta/ts6xnrp1/hssh/a2cuerht/s...,0
4,Valid,/2m6vlb1r37jspc/cwvv/mbar/oqrd0/msc/etceebwgi/...,0
...,...,...,...
125971,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125972,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125973,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0
125974,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0


In [ ]:
# @title Gán thuộc tính cho từng dòng

# @markdown Tạo một list chứa các thuộc tính. Thêm các thuộc tính này thành các cột trong DataFrame sau đó gán nhãn 1 cho những dòng có url chứa thuộc tính tương ứng


ls = ["../", "..\\", "etc", "passwd", "\\.", "\\/", "./", "/", ":", "//", ":/",
      "system", "ini", "..", "exec", ":\\", "%00", ".bat", "file", "windows",
      "boot", "winnt", ".conf", "access", "log", ",,"]

attr = set(ls)
len(attr)

for a in attr:
  df[a] = 0

def generate(statement,col):
    if col in str(statement):
        return 1
    else:
        return 0

for col in attr:
  df[col] = df['url'].apply(generate, col=col)

df

,type,url,label,"""",xp,all,=,any,+,>,...,:,like,-,union,(,char,!=,&&,update,commit
0,Valid,/l_t@/_feu1wvhtpass2/1nieqnnrvnzktuasain/tg1ar...,0,0,0,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
1,Valid,/inssgtz7ieltdsstbw7e/neqhmsdwu7imdb0etet/et/h...,0,0,1,0,1,0,1,0,...,0,0,1,0,0,0,0,0,0,0
2,Valid,/fonrnt/7n.d-4brssxb@tu/qghew.cfm,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Valid,/dyylkl.xd9cpu/4ot0ta/ts6xnrp1/hssh/a2cuerht/s...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Valid,/2m6vlb1r37jspc/cwvv/mbar/oqrd0/msc/etceebwgi/...,0,0,0,0,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125971,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125972,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125973,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
125974,Valid,/antoanweb/publico/registro.jsp?modo=registro&...,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# @title Xuất df ra thành final_PT.csv

import csv
df.to_csv('/content/drive/MyDrive/Machine_Log_Analysis/Dataset/Final_Csv/final_PT.csv', escapechar='\\', quoting=csv.QUOTE_MINIMAL)

# Thực hiện tạo dataset OSCommanding

In [ ]:
# @title Đọc stage.csv vào DataFrame df và thêm cột label


df = pd.read_csv("/content/drive/MyDrive/Machine_Log_Analysis/Dataset/stage.csv")
df = df[['type','url']]
df['label'] = 0
df

In [ ]:
# @title Gán nhãn label cho các hàng OS và chuẩn hóa URL

# @markdown - Thực hiện:
# @markdown - Gán nhãn những hàng có type = OS với label = 1
# @markdown - Fill những ô trống bằng NULL
# @markdown - Chuẩn hóa url: unquote, bỏ các giá trị newline, khoảng trống và đưa tất cả về chữ thường


df.loc[df['type']=='OS','label'] = 1
df = df.fillna('NULL')

df['url'] = df['url'].apply(url_parse)
df['url'] = df['url'].apply(newremove)
df['url'] = df['url'].apply(plusreplace)

df['url'] = df['url'].str.lower()

In [ ]:
# @title Gán thuộc tính cho từng dòng

# @markdown Tạo một list chứa các thuộc tính. Thêm các thuộc tính này thành các cột trong DataFrame sau đó gán nhãn 1 cho những dòng có url chứa thuộc tính tương ứng

ls = ["../", "..\\", "etc", "passwd", "\\.", "\\/", "./", ":", ":/", ".",
      "system32", "display", ".exe", "cmd", "dir", ";", "tmp/", "etc/passwd",
      "wget", "cat", "ping", "bash", "ftp", "|", "..", "exec", ":\\", ".bat",
      "file", "script", "rm", "c:", "winnt", "access", "log", "'", "www.",
      "http", "bin/", "telnet", "echo", "root", "-aux", "shell", "uname", "IP"]

attr = set(ls)
len(attr)

for a in attr:
  df[a] = 0

def generate(statement,col):
    if col in str(statement):
        return 1
    else:
        return 0

for col in attr:
  df[col] = df['url'].apply(generate, col=col)

df

In [ ]:
# @title Xuất df ra thành final_OS.csv

import csv
df.to_csv('/content/drive/MyDrive/Machine_Log_Analysis/Dataset/Final_Csv/final_OS.csv', escapechar='\\', quoting=csv.QUOTE_MINIMAL)